# 로지스틱 & 소프트맥스 잠시 복습하기
- 로지스틱은 시그모이드 함수를 통해서 P(x=1)일 확률을 구해주는 형태로 학습이 진행된다.
- 따라서 우리가 정한 Threshold를 넘어서면, 해당 class에 속한다고 판단가능하다.
- Binary cross-entropy 기법을 사용한다.


In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [28]:
# 데이터정의하기
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]] # 6 X 2
y_data = [[0],[0],[0],[1],[1],[1]] # 6 X 1

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [32]:
# 모델만들기
class Logistic_Model(nn.Module):
    def __init__(self):
        super().__init__()
#         self.linear = nn.Linear(2,1)
#         self.sigmoid = nn.Sigmoid()
        self.layer = nn.Sequential(
            nn.Linear(2,1),
            nn.Sigmoid(),
        )
        
    def forward(self,x):
        #return self.sigmoid(self.linear(x))
        return self.layer(x)
    
L_model = Logistic_Model()

In [33]:
#optim
optimizer = optim.SGD(L_model.parameters(), lr = 1)

In [34]:
# training
n_epoch = 100
for epoch in range(n_epoch+1):
    #H(x)계산하기
    hypothesis = L_model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    #backprop
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #출력하기
    if epoch % 10 == 0:
        pred = hypothesis > torch.FloatTensor([0.5])
        acc = (pred.float() == y_train).sum().item() / len(pred)
        print("Epoch {:4d}/{} Cost : {} acc:{}".format(epoch, n_epoch, cost.item(),acc*100))

Epoch    0/100 Cost : 0.5767403244972229 acc:83.33333333333334
Epoch   10/100 Cost : 0.8181087970733643 acc:66.66666666666666
Epoch   20/100 Cost : 0.5821924209594727 acc:83.33333333333334
Epoch   30/100 Cost : 0.48843690752983093 acc:83.33333333333334
Epoch   40/100 Cost : 0.40266355872154236 acc:83.33333333333334
Epoch   50/100 Cost : 0.3210720121860504 acc:83.33333333333334
Epoch   60/100 Cost : 0.2470589280128479 acc:83.33333333333334
Epoch   70/100 Cost : 0.19001829624176025 acc:100.0
Epoch   80/100 Cost : 0.15886469185352325 acc:100.0
Epoch   90/100 Cost : 0.14483368396759033 acc:100.0
Epoch  100/100 Cost : 0.13489697873592377 acc:100.0


# Soft max

In [37]:
# 데이터 정의하기
# 8 x 4
x_train = [[1,2,1,1],
           [2,1,3,2],
           [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
           [1,2,5,6],
           [1,6,6,6], 
           [1,7,7,7]]
# (8,)의 형태를 가진다.
y_train = [2,2,2,1,1,1,0,0] # 3개의 클래스에서 index를 의미한다. 예를들어 2를 사용하면 (0,0,1)
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [55]:
# model 정의하기
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(4,3),
        )
        
    def forward(self,x):
        return self.layer(x)

S_model = SoftmaxClassifierModel()

#optimizer
optimizer = optim.SGD(S_model.parameters(), lr = 1e-1)

In [56]:
# train

n_epoch = 1000
for epoch in range(n_epoch+1):
    #H(x)계산하기
    hypothesis = S_model(x_train)
    cost = F.cross_entropy(hypothesis, y_train)
    
    #backprop
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #출력하기
    if epoch % 100 == 0:
        #pred = hypothesis > torch.FloatTensor([0.5])
        #acc = (pred.float() == y_train).sum().item() / len(pred)
        pred = torch.argmax(F.softmax(hypothesis, dim = 1), dim = 1)
        acc = (pred == y_train).sum().item() / len(pred)
        print("Epoch {:4d}/{} Cost : {} acc:{}".format(epoch, n_epoch, cost.item(),acc*100))

Epoch    0/1000 Cost : 1.7697954177856445 acc:25.0
Epoch  100/1000 Cost : 0.6833751201629639 acc:75.0
Epoch  200/1000 Cost : 0.6049606204032898 acc:75.0
Epoch  300/1000 Cost : 0.5480685234069824 acc:75.0
Epoch  400/1000 Cost : 0.4976436197757721 acc:87.5
Epoch  500/1000 Cost : 0.44987839460372925 acc:87.5
Epoch  600/1000 Cost : 0.4032818078994751 acc:87.5
Epoch  700/1000 Cost : 0.35716378688812256 acc:87.5
Epoch  800/1000 Cost : 0.31143900752067566 acc:87.5
Epoch  900/1000 Cost : 0.26794832944869995 acc:87.5
Epoch 1000/1000 Cost : 0.23981989920139313 acc:100.0


# 느낀점

- clasification 모델에 대해서 배울 수 있었다.
- 로지스틱은 시그모이드 함수를 통해서, P(x=1)인 확률을 계산하고, binary cross entropy 사용한다.

- 소프트맥스는 소프트맥스함수를 이용해서 class별 확률을 계산하고, cross_entropy 함수를 사용한다.

# 오늘의 수업 Tips

In [101]:
x_train = torch.FloatTensor([[1,2,1],
                             [1,3,2],
                             [1,3,4],
                             [1,5,5],
                             [1,7,5],
                             [1,2,5],
                             [1,6,6],
                             [2,7,7]
                            ])
y_train = torch.LongTensor([2,2,2,1,1,1,0,0])

In [102]:
x_test = torch.FloatTensor([[2,1,1],[3,1,2],[3,3,4]])
y_test = torch.LongTensor([2,2,2])

In [103]:
#model
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,3)
        
    def forward(self,x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

#optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.1)


In [86]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction  = model(x_train)
        cost = F.cross_entropy(prediction, y_train)
        
        # cost계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost:{:.6f}'.format(epoch,nb_epochs,cost.item()))
        
        

In [87]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    
    cost = F.cross_entropy(prediction, y_test)
    
    print('Accuracy:{}% Cost:{:.6f}'.format(correct_count/len(y_test)*100, cost.item()))
    

# Learning rate가 클때와 작을때

In [91]:
model = SoftmaxClassifierModel()

#optimizer
optimizer = optim.SGD(model.parameters(), lr = 1e5) # 클떄
train(model, optimizer, x_train, y_train)


Epoch    0/20 Cost:1.826495
Epoch    1/20 Cost:1252283.500000
Epoch    2/20 Cost:1883201.250000
Epoch    3/20 Cost:741362.125000
Epoch    4/20 Cost:1941346.000000
Epoch    5/20 Cost:275388.781250
Epoch    6/20 Cost:928846.000000
Epoch    7/20 Cost:2564799.500000
Epoch    8/20 Cost:492908.437500
Epoch    9/20 Cost:1750737.125000
Epoch   10/20 Cost:728513.750000
Epoch   11/20 Cost:1181971.000000
Epoch   12/20 Cost:1872612.125000
Epoch   13/20 Cost:746033.437500
Epoch   14/20 Cost:1093121.875000
Epoch   15/20 Cost:970701.250000
Epoch   16/20 Cost:1413221.000000
Epoch   17/20 Cost:1347612.125000
Epoch   18/20 Cost:983649.562500
Epoch   19/20 Cost:631366.000000


In [93]:
model = SoftmaxClassifierModel()

#optimizer
optimizer = optim.SGD(model.parameters(), lr = 1e-5) # 작을때
train(model, optimizer, x_train, y_train)


Epoch    0/20 Cost:2.869081
Epoch    1/20 Cost:2.868805
Epoch    2/20 Cost:2.868531
Epoch    3/20 Cost:2.868255
Epoch    4/20 Cost:2.867980
Epoch    5/20 Cost:2.867705
Epoch    6/20 Cost:2.867430
Epoch    7/20 Cost:2.867155
Epoch    8/20 Cost:2.866880
Epoch    9/20 Cost:2.866605
Epoch   10/20 Cost:2.866330
Epoch   11/20 Cost:2.866055
Epoch   12/20 Cost:2.865780
Epoch   13/20 Cost:2.865505
Epoch   14/20 Cost:2.865231
Epoch   15/20 Cost:2.864956
Epoch   16/20 Cost:2.864681
Epoch   17/20 Cost:2.864406
Epoch   18/20 Cost:2.864131
Epoch   19/20 Cost:2.863857


# Data Preprocessing

- Standard deviation 구하기

In [131]:
mu = x_train.mean(dim = 0) # dim 0을 죽임
sigma = x_train.std(dim = 0)

In [148]:
x_train

tensor([[1., 2., 1.],
        [1., 3., 2.],
        [1., 3., 4.],
        [1., 5., 5.],
        [1., 7., 5.],
        [1., 2., 5.],
        [1., 6., 6.],
        [2., 7., 7.]])

In [147]:
mu

tensor([1.1250, 4.3750, 4.3750])

In [132]:
norm_x_train = (x_train - mu) / sigma

In [133]:
print(norm_x_train)

tensor([[-0.3536, -1.1130, -1.6913],
        [-0.3536, -0.6444, -1.1902],
        [-0.3536, -0.6444, -0.1879],
        [-0.3536,  0.2929,  0.3132],
        [-0.3536,  1.2301,  0.3132],
        [-0.3536, -1.1130,  0.3132],
        [-0.3536,  0.7615,  0.8143],
        [ 2.4749,  1.2301,  1.3154]])


In [139]:
norm_x_train.dtype
y_train = y_train.float()

In [140]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self,x):
        return self.linear(x)
model = MultivariateLinearRegressionModel()

In [144]:
optimizer = optim.SGD(model.parameters(), lr = 1e-1)

In [145]:
#이번에는 regression 모델을 만듭시다.
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction  = model(x_train)
        cost = F.mse_loss(prediction, y_train)
        
        # cost계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost:{:.6f}'.format(epoch,nb_epochs,cost.item()))

In [146]:
train(model,optimizer,norm_x_train,y_train)

Epoch    0/20 Cost:1.438417
Epoch    1/20 Cost:1.052675
Epoch    2/20 Cost:0.866802
Epoch    3/20 Cost:0.769861
Epoch    4/20 Cost:0.715177
Epoch    5/20 Cost:0.682128
Epoch    6/20 Cost:0.661016
Epoch    7/20 Cost:0.646945
Epoch    8/20 Cost:0.637256
Epoch    9/20 Cost:0.630410
Epoch   10/20 Cost:0.625468
Epoch   11/20 Cost:0.621836
Epoch   12/20 Cost:0.619122
Epoch   13/20 Cost:0.617066
Epoch   14/20 Cost:0.615488
Epoch   15/20 Cost:0.614264
Epoch   16/20 Cost:0.613305
Epoch   17/20 Cost:0.612548
Epoch   18/20 Cost:0.611946
Epoch   19/20 Cost:0.611464


<ipython-input-145-b54eb8bdb73e>:6: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = F.mse_loss(prediction, y_train)


In [154]:
x_train.mean(dim=0)

tensor([1.1250, 4.3750, 4.3750])

In [153]:
x_train.mean(dim=1)

tensor([1.3333, 2.0000, 2.6667, 3.6667, 4.3333, 2.6667, 4.3333, 5.3333])